In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [ ]:
URL = "https://travelness.com/most-visited-cities-in-the-world"

In [ ]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geolocator = Nominatim(user_agent="geocode_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)

In [ ]:
webpage = requests.get(URL, headers=HEADER)
webpage

In [ ]:
soup = BeautifulSoup(webpage.content, "html.parser")

In [ ]:
data = soup.find_all("td", attrs = {'class': 'has-text-align-left'})
len(data)

In [ ]:
print(data[0].text, data[1].text, data[2].text)
print(data[-3].text, data[-2].text, data[-1].text)
# Use i % 3 == {0,1,2} in if-else for different data

In [ ]:
location_description2 = geocoder.geocode(data[4].text)
len(location_description2)

In [ ]:
location = geolocator.geocode(data[7].text, language="en")
print(location)
print(data[7].text)
latitude = str(location.raw['lat'])
longitude = str(location.raw['lon'])
lat_long = latitude + ","+ longitude 
location2 = geolocator.reverse(lat_long, language="en")
city = data[7].text
try:
    postcode = location2.raw['address']['postcode'].replace(" ","")
except:
    postcode = "Null"
country = location2.raw['address']['country'].upper()
country_code = location2.raw['address']['country_code']
print(city + "  " + country + "  "+ str(round(float(latitude),2)) + "  "+ str(round(float(longitude),2)) + "  " + country_code.upper() + "  " + postcode)
#location2.raw

In [ ]:
city_address = []
for i in range(60):
    if i % 3 == 1:

        # GETTING RAW DATA
        try:
            location_description = geocoder.geocode(data[i].text)
        except Exception:
            print(str(i) + " " + "No data found for location: " + str(data[i].text))
            continue
        
        # GETTING ADDRESS COMPONENTS
        city = data[i].text
        country = location_description[0]['components']['country']
        country_code = location_description[0]['components']['country_code']
        continent = location_description[0]['components']['continent']
        latitude = str(location_description[0]['geometry']['lat'])
        longitude = str(location_description[0]['geometry']['lng'])
        visitors = data[i+1].text.split(" ")[0]

        #GETTING POSTCODE
        lat_long = str(latitude) + ","+ str(longitude) 
        location2 = geolocator.reverse(lat_long, language="en")
        try:
            postcode = location2.raw['address']['postcode'].replace(" ","")
        except:
            postcode = "NULL"
        
        # ADDING VARIABLES TO DICTIONARY
        #print(str(i) +"  "+ city +"  " + country + "  " + country_code.upper() + "  " + continent + "  " + latitude + "  "+ longitude+ "  " + postcode + "  " + visitors)
        address_dict = {'City': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode, 'Visitors (in Millions)': visitors}
        city_address.append(address_dict)
    
print(city_address)

In [ ]:
#city_address = []
#for i in range(60):
#    if i % 3 == 1:
#
#        # GETTING LATITUDE & LONGITUDE
#        location = geolocator.geocode(data[i].text, language="en")
#        latitude = str(location.raw['lat'])
#        longitude = str(location.raw['lon'])
#        lat_long = latitude + ","+ longitude 
#        location2 = geolocator.reverse(lat_long, language="en")
#        
#        # ASSIGNING VARIABLES
#        city = data[i].text
#        country = location2.raw['address']['country']
#        country_code = location2.raw['address']['country_code']
#        try:
#            postcode = location2.raw['address']['postcode'].replace(" ","-")
#        except:
#            postcode = "NULL"
#        visitors = data[i+1].text.split(" ")[0]
#        
#        # ADDING VARIABLES TO DICTIONARY
#        # print(str(i) +"  "+ city +"  " + country + "  " + country_code.upper() + "  " + latitude + "  "+ longitude+ "  " + postcode + "  " + visitors)
#        address_dict = {'City': city, 'Country': country, 'Country Code':country_code.upper(), 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode, 'Visitors (in Millions)': visitors}
#        city_address.append(address_dict)
#    
#print(city_address)

In [ ]:
#ca_most_tourist_df_2 = pd.DataFrame.from_dict(city_address_2)
#ca_most_tourist_df_2

In [ ]:
ca_most_tourist_df = pd.DataFrame.from_dict(city_address)
ca_most_tourist_df